In [1]:
import sys
import os
from baselines import logger, bench
from baselines.common import set_global_seeds
from baselines.common.cmd_util import make_atari_env, atari_arg_parser
from baselines.common.atari_wrappers import wrap_deepmind, make_atari
from baselines.common.vec_env.vec_frame_stack import VecFrameStack
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
from baselines.common.vec_env.vec_normalize import VecNormalize, VecNormalizeTest
from baselines.ppo2 import ppo2
#from baselines.ppo2.policies import CnnPolicy, LstmPolicy, LnLstmPolicy, MlpPolicy
import multiprocessing
import tensorflow as tf
import pandas as pd
import numpy as np
import gym

from src.common.environments import ESTradingEnv

import matplotlib as mpl
%matplotlib

/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Using matplotlib backend: Qt5Agg


/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/site-packages/matplotlib/cbook/deprecation.py:106: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


In [2]:
#Register trading environment with OpenAI Gym
from gym.envs.registration import register
register(id='tradingenv-v0',
         entry_point='src.common.environments:ESTradingEnv_v2',)

In [3]:
from baselines.a2c.utils import fc, conv_to_fc, lstm, batch_to_seq, seq_to_batch
from baselines.common.distributions import make_pdtype
from baselines.common.input import observation_input

def ortho_init(scale=1.0):
    def _ortho_init(shape, dtype, partition_info=None):
        #lasagne ortho init for tf
        shape = tuple(shape)
        if len(shape) == 2:
            flat_shape = shape
        elif len(shape) > 2: # assumes NHWC
            flat_shape = (np.prod(shape[:-1]), shape[-1])
        else:
            raise NotImplementedError
        a = np.random.normal(0.0, 1.0, flat_shape)
        u, _, v = np.linalg.svd(a, full_matrices=False)
        q = u if u.shape == flat_shape else v # pick the one with the correct shape
        q = q.reshape(shape)
        return (scale * q[:shape[0], :shape[1]]).astype(np.float32)
    return _ortho_init

def conv(x, scope, *, nf, rf, stride, pad='VALID', init_scale=1.0, one_dim_bias=False):    
    channel_ax = 2
    strides = [1, stride, 1]
    bshape = [1, 1, nf]
    
    nin = x.get_shape()[channel_ax].value
    wshape = [rf, nin, nf]
    with tf.variable_scope(scope):
        w = tf.get_variable("w", wshape, initializer=ortho_init(init_scale))
        b = tf.get_variable("b", bshape, initializer=tf.constant_initializer(0.0))
        return b + tf.nn.conv1d(x, w, stride=stride, padding=pad)


def cnn(X, is_training, **conv_kwargs):
    activ = tf.nn.relu
    h = activ(conv(X, 'c1', nf=16, rf=4, stride=1, init_scale=np.sqrt(2),
                   **conv_kwargs))
    h = tf.layers.dropout(h, rate=0.5, training=is_training)
    h2 = activ(conv(h, 'c2', nf=8, rf=4, stride=1, init_scale=np.sqrt(2), **conv_kwargs))
    h2 = tf.layers.dropout(h2, rate=0.5, training=is_training)
    h3 = conv_to_fc(h2)
    return activ(fc(h3, 'fc1', nh=32, init_scale=np.sqrt(2)))


class CnnPolicy(object):

    def __init__(self, sess, ob_space, ac_space, nbatch, nsteps, reuse=False, **conv_kwargs): #pylint: disable=W0613
        if nsteps == 1:
            self._is_training = False
        else:
            self._is_training = True
            
        self.pdtype = make_pdtype(ac_space)
        X, processed_x = observation_input(ob_space, nbatch)
        with tf.variable_scope("model", reuse=reuse):
            h = cnn(processed_x, self._is_training, **conv_kwargs)
            #activ = tf.nn.relu
            #print(processed_x.get_shape())
            #h = activ(fc(processed_x, 'fc1', nh=512, init_scale=np.sqrt(2)))
            vf = fc(h, 'v', 1)[:,0]
            self.pd, self.pi = self.pdtype.pdfromlatent(h, init_scale=0.01)

        a0 = self.pd.sample()
        neglogp0 = self.pd.neglogp(a0)
        self.initial_state = None

        def step(ob, *_args, **_kwargs):
            a, v, neglogp = sess.run([a0, vf, neglogp0], {X:ob})
            return a, v, self.initial_state, neglogp

        def value(ob, *_args, **_kwargs):
            return sess.run(vf, {X:ob})

        self.X = X
        self.vf = vf
        self.step = step
        self.value = value
        
class LstmPolicy(object):

    def __init__(self, sess, ob_space, ac_space, nbatch, nsteps, nlstm=256, reuse=False):
        if nsteps == 1:
            self._is_training = False
        else:
            self._is_training = True
            
        nenv = nbatch // nsteps
        self.pdtype = make_pdtype(ac_space)
        X, processed_x = observation_input(ob_space, nbatch)

        M = tf.placeholder(tf.float32, [nbatch]) #mask (done t-1)
        S = tf.placeholder(tf.float32, [nenv, nlstm*2]) #states
        with tf.variable_scope("model", reuse=reuse):
            h = cnn(processed_x, self._is_training)
            xs = batch_to_seq(h, nenv, nsteps)
            ms = batch_to_seq(M, nenv, nsteps)
            h5, snew = lstm(xs, ms, S, 'lstm1', nh=nlstm)
            h5 = tf.layers.dropout(h5, rate=0.5, training=self._is_training)
            h5 = seq_to_batch(h5)
            vf = fc(h5, 'v', 1)
            self.pd, self.pi = self.pdtype.pdfromlatent(h5)

        v0 = vf[:, 0]
        a0 = self.pd.sample()
        neglogp0 = self.pd.neglogp(a0)
        self.initial_state = np.zeros((nenv, nlstm*2), dtype=np.float32)

        def step(ob, state, mask):
            return sess.run([a0, v0, snew, neglogp0], {X:ob, S:state, M:mask})

        def value(ob, state, mask):
            return sess.run(v0, {X:ob, S:state, M:mask})

        self.X = X
        self.M = M
        self.S = S
        self.vf = vf
        self.step = step
        self.value = value

In [4]:
def train(env_id, num_timesteps, seed, policy):

    ncpu = multiprocessing.cpu_count()
    if sys.platform == 'darwin': ncpu //= 2
    config = tf.ConfigProto(allow_soft_placement=True,
                            intra_op_parallelism_threads=ncpu,
                            inter_op_parallelism_threads=ncpu)
    config.gpu_options.allow_growth = True #pylint: disable=E1101
    tf.Session(config=config).__enter__()
    
    def make_env(rank):
        def _thunk():
            env = gym.make(env_id)
            env.seed(seed + rank)
            env = bench.Monitor(env, logger.get_dir() and os.path.join(logger.get_dir(), str(rank)))
            return env
        return _thunk
    

    #env = DummyVecEnv([make_env])
    env = SubprocVecEnv([make_env(i) for i in range(32)])
    env = VecNormalize(env, ob=True)
    set_global_seeds(seed)

    policy = LstmPolicy
    ppo2.learn(policy=policy, env=env, nsteps=128, nminibatches=4,
        lam=0.95, gamma=0.99, noptepochs=25, log_interval=1,
        ent_coef=.01,
        lr=lambda f: f * 2.5e-4,
        cliprange=0.1,
        total_timesteps=int(num_timesteps * 1.1))

In [5]:
logger.configure()

train('tradingenv-v0', num_timesteps=1e7, seed=0, policy='cnn')

Logging to /tmp/openai-2018-05-30-18-51-01-525742
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
--------------------------------------
| approxkl           | 0.000938044   |
| clipfrac           | 0.021708984   |
| eplenmean          | 96.5          |
| eprewmean          | -523          |
| explained_variance | -0.297        |
| fps                | 638           |
| nupdates           | 1             |
| policy_entropy     | 1.0976362     |
| policy_loss        | -0.0011973628 |
| serial_timesteps   | 128           |
| time_elapsed       | 6.41          |
| total_timesteps    | 4096          |
| value_loss         | 0.6793031     |
--------------------------------------
Saving to /tmp/openai-2018-

--------------------------------------
| approxkl           | 0.0020687236  |
| clipfrac           | 0.07727539    |
| eplenmean          | 204           |
| eprewmean          | -525          |
| explained_variance | 0.0157        |
| fps                | 768           |
| nupdates           | 14            |
| policy_entropy     | 1.0594614     |
| policy_loss        | -0.0006887268 |
| serial_timesteps   | 1792          |
| time_elapsed       | 74.8          |
| total_timesteps    | 57344         |
| value_loss         | 0.05905364    |
--------------------------------------
---------------------------------------
| approxkl           | 0.0018217039   |
| clipfrac           | 0.059023436    |
| eplenmean          | 221            |
| eprewmean          | -524           |
| explained_variance | 0.00348        |
| fps                | 775            |
| nupdates           | 15             |
| policy_entropy     | 1.0548551      |
| policy_loss        | -0.00042630904 |
| serial_timest

---------------------------------------
| approxkl           | 0.0017026842   |
| clipfrac           | 0.047060546    |
| eplenmean          | 249            |
| eprewmean          | -524           |
| explained_variance | 0.00729        |
| fps                | 792            |
| nupdates           | 28             |
| policy_entropy     | 1.0131191      |
| policy_loss        | -0.00018720294 |
| serial_timesteps   | 3584           |
| time_elapsed       | 147            |
| total_timesteps    | 114688         |
| value_loss         | 0.059608735    |
---------------------------------------
--------------------------------------
| approxkl           | 0.0020274615  |
| clipfrac           | 0.081318356   |
| eplenmean          | 241           |
| eprewmean          | -524          |
| explained_variance | 0.0169        |
| fps                | 770           |
| nupdates           | 29            |
| policy_entropy     | 1.0051094     |
| policy_loss        | -0.0011492603 |
| serial_t

-------------------------------------
| approxkl           | 0.001771481  |
| clipfrac           | 0.056962892  |
| eplenmean          | 257          |
| eprewmean          | -526         |
| explained_variance | 0.0319       |
| fps                | 769          |
| nupdates           | 43           |
| policy_entropy     | 0.99385065   |
| policy_loss        | 0.0006580582 |
| serial_timesteps   | 5504         |
| time_elapsed       | 225          |
| total_timesteps    | 176128       |
| value_loss         | 0.062310237  |
-------------------------------------
-------------------------------------
| approxkl           | 0.0021229656 |
| clipfrac           | 0.08518555   |
| eplenmean          | 268          |
| eprewmean          | -527         |
| explained_variance | -0.014       |
| fps                | 757          |
| nupdates           | 44           |
| policy_entropy     | 0.9960192    |
| policy_loss        | -0.000622154 |
| serial_timesteps   | 5632         |
| time_elaps

---------------------------------------
| approxkl           | 0.0021399246   |
| clipfrac           | 0.078525394    |
| eplenmean          | 268            |
| eprewmean          | -527           |
| explained_variance | 0.00465        |
| fps                | 781            |
| nupdates           | 57             |
| policy_entropy     | 0.9422295      |
| policy_loss        | -0.00026629522 |
| serial_timesteps   | 7296           |
| time_elapsed       | 300            |
| total_timesteps    | 233472         |
| value_loss         | 0.0663784      |
---------------------------------------
---------------------------------------
| approxkl           | 0.001646881    |
| clipfrac           | 0.0525         |
| eplenmean          | 286            |
| eprewmean          | -526           |
| explained_variance | 0.00973        |
| fps                | 803            |
| nupdates           | 58             |
| policy_entropy     | 0.9511803      |
| policy_loss        | -0.00019617962 |


...

  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/repos/baselines/baselines/common/vec_env/subproc_vec_env.py", line 10, in worker
    cmd, data = remote.recv()
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/repos/baselines/baselines/common/vec_env/subproc_vec_env.py", line 10, in worker
    cmd, data = remote.recv()
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  

  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/ubuntu/repos/baselines/baselines/common/vec_env/subproc_vec_env.py", line 10, in worker
    cmd, data = remote.recv()
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/home/ubuntu/projects/algotrading/notebooks/src/common/environments.py", line 207, in step
    position=self._position)
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/

  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py", line 1325, in __getitem__
    return self._getitem_tuple(key)
  File "/home/ubuntu/src/anaconda3/envs/tenso

KeyboardInterrupt: 

  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
  File "/home/ubuntu/src/anaconda3/envs/tensorflow/lib/python3.6/site-packages/pandas/core/indexing.py", line 189, in _has

In [6]:
load_path = '/tmp/openai-2018-05-29-15-08-04-310254/checkpoints/00100'
logger.get_dir()

'/tmp/openai-2018-05-29-15-08-04-310254'

In [4]:
def test(env_id, load_path, seed):

    ncpu = multiprocessing.cpu_count()
    if sys.platform == 'darwin': ncpu //= 2
    config = tf.ConfigProto(allow_soft_placement=True,
                            intra_op_parallelism_threads=ncpu,
                            inter_op_parallelism_threads=ncpu)
    config.gpu_options.allow_growth = True #pylint: disable=E1101
    tf.Session(config=config).__enter__()
    
    def make_env():
        env = gym.make(env_id)
        return env
    
    
    env = DummyVecEnv([make_env])
    running_mean = np.load('{}/mean.npy'.format(load_path))
    running_var = np.load('{}/var.npy'.format(load_path))
    env = VecNormalizeTest(env, running_mean, running_var)
    set_global_seeds(seed)

    policy = CnnPolicy
    ob_space = env.venv.observation_space
    ac_space = env.venv.action_space
    nbatch_act = 1 * 1
    nbatch_train = 32 * 1
    nsteps = 128
    ent_coef = .01
    vf_coef = 0.5
    max_grad_norm = 0.5
    make_model = lambda : ppo2.Model(policy=policy, ob_space=ob_space, ac_space=ac_space, 
                                nbatch_act=nbatch_act, nbatch_train=nbatch_train, nsteps=nsteps, 
                                ent_coef=ent_coef, vf_coef=vf_coef, max_grad_norm=max_grad_norm)
    model = make_model()
    model.load(load_path + '/checkpoints/02150')
    
    obs = env.reset()
    while True:
        actions, _, _, _, = model.step(obs)
        obs[:], _, _, _, = env.step(actions)
        env.venv.render()

In [ ]:
load_path = '../data/processed'
load_path = '/tmp/openai-2018-05-30-18-51-01-525742'
test('tradingenv-v0', load_path, seed=0)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
def make_env():
        env = gym.make('tradingenv-v0')
        return env
env = DummyVecEnv([make_env])
load_path = '/tmp/openai-2018-05-29-15-08-04-310254'
running_mean = np.load('{}/mean.npy'.format(load_path))
running_var = np.load('{}/var.npy'.format(load_path))
env = VecNormalizeTest(env, running_mean, running_var)

In [3]:
import gym
e = gym.make('tradingenv-v5')

In [13]:
e.step(1)

(array([[ 2.44675000e+03,  2.44800000e+03,  2.44675000e+03,
          2.44725000e+03, -8.43391446e-01, -5.37299608e-01,
          0.00000000e+00],
        [ 2.44725000e+03,  2.44800000e+03,  2.44675000e+03,
          2.44700000e+03, -8.54911871e-01, -5.18773258e-01,
          0.00000000e+00],
        [ 2.44700000e+03,  2.44775000e+03,  2.44675000e+03,
          2.44700000e+03, -8.66025404e-01, -5.00000000e-01,
          0.00000000e+00],
        [ 2.44725000e+03,  2.44875000e+03,  2.44700000e+03,
          2.44775000e+03, -8.76726756e-01, -4.80988769e-01,
          0.00000000e+00],
        [ 2.44800000e+03,  2.44925000e+03,  2.44775000e+03,
          2.44850000e+03, -8.87010833e-01, -4.61748613e-01,
          0.00000000e+00],
        [ 2.44850000e+03,  2.44950000e+03,  2.44850000e+03,
          2.44900000e+03, -8.96872742e-01, -4.42288690e-01,
          0.00000000e+00],
        [ 2.44900000e+03,  2.44950000e+03,  2.44850000e+03,
          2.44850000e+03, -9.06307787e-01, -4.22618262e-01